In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.7 MB/s eta 0:00:00


In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.4 MB/s eta 0:00:00


In [ ]:
!pip install accelerate -U

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding

In [ ]:
from datasets import load_dataset

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
raw_datasets=load_dataset('glue','mrpc')

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [ ]:
for i in range(5):
  print(raw_datasets['validation']['sentence1'][i])
  print(raw_datasets['validation']['sentence2'][i])
  print(raw_datasets['train']['label'][i])


He said the foodservice pie business doesn 't fit the company 's long-term growth strategy .
" The foodservice pie business does not fit our long-term growth strategy .
1
Magnarelli said Racicot hated the Iraqi regime and looked forward to using his long years of training in the war .
His wife said he was " 100 percent behind George Bush " and looked forward to using his years of training in the war .
0
The dollar was at 116.92 yen against the yen , flat on the session , and at 1.2891 against the Swiss franc , also flat .
The dollar was at 116.78 yen JPY = , virtually flat on the session , and at 1.2871 against the Swiss franc CHF = , down 0.1 percent .
1
The AFL-CIO is waiting until October to decide if it will endorse a candidate .
The AFL-CIO announced Wednesday that it will decide in October whether to endorse a candidate before the primaries .
0
No dates have been set for the civil or the criminal trial .
No dates have been set for the criminal or civil cases , but Shanley has ple

In [ ]:
checkpoint='bert-base-uncased'

In [ ]:
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenize_function(example):
  return tokenizer(example['sentence1'],example['sentence2'], truncation=True)


In [ ]:
tokenized_dataset=raw_datasets.map(tokenize_function,batched=True)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [ ]:
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
training_args=TrainingArguments("test-trainer")

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

Step,Training Loss
500,0.108100
1000,0.044700


TrainOutput(global_step=1377, training_loss=0.05955576186779191, metrics={'train_runtime': 203.4792, 'train_samples_per_second': 54.079, 'train_steps_per_second': 6.767, 'total_flos': 405626802939840.0, 'train_loss': 0.05955576186779191, 'epoch': 3.0})

In [ ]:
trainer.save_model('/content/model')

In [ ]:
tokenizer.save_pretrained('/content/model')

('/content/model/tokenizer_config.json',
 '/content/model/special_tokens_map.json',
 '/content/model/vocab.txt',
 '/content/model/added_tokens.json',
 '/content/model/tokenizer.json')

In [ ]:
mkdir /content/model

In [ ]:
import evaluate
import numpy as np
def compute_metric(values):
  metric=evaluate.load("glue","mrpc")
  logits,labels=values
  preds=np.argmax(logits,axis=-1)
  return metric.compute(predictions=preds,reference=labels)

In [ ]:
sentence1 = "The quick brown fox jumps over the lazy dog ."
sentence2="A fast, brown fox leaps over a sleepy dog ."
sentence_encoded1=tokenizer(sentence2,sentence1,padding=True, truncation=True, max_length=128,return_tensors="pt")
#sentence_encoded2=tokenizer(sentence2,padding=True, truncation=True, max_length=128,return_tensors="pt")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('/content/model')
tokenizer = AutoTokenizer.from_pretrained('/content/model')

In [ ]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
trainer.model

In [ ]:
sentence_encoded = {k: v.to(device) for k, v in sentence_encoded1.items()}


In [ ]:
import torch
with torch.no_grad():
  output=model(**sentence_encoded)
  logits = output.logits
  probabilities = torch.nn.functional.softmax(logits, dim=-1)
  print("Probs=",probabilities)
  predicted_class = torch.argmax(probabilities, dim=-1)
  predicted_class_index = predicted_class.item()
  class_names = ["Not", "Paraphrase"]
  predicted_class_name = class_names[predicted_class_index]
  print(f"Predicted class: {predicted_class_name}")


Probs= tensor([[1.9722e-05, 9.9998e-01]], device='cuda:0')
Predicted class: Paraphrase


In [ ]:
print(logits)

tensor([[ 4.7017, -4.4944]], device='cuda:0')


In [ ]:
class_names = ["Paraphrase", "Not"]
predicted_class_name = class_names[predicted_class_index]
print(f"Predicted class: {predicted_class_name}")

Predicted class: Paraphrase
